# Uk Parliament Debate Files - infformation retrieval and processing

### Matan  Aminoff 326636545, Noam Solow - 214928517

## Section A: Download Debate Files§

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from tqdm import tqdm

def download_debate_files():
    """
    Downloads UK Parliament debate files from the specified start file.
    """
    base_url = "https://www.theyworkforyou.com/pwdata/scrapedxml/debates/"  # 
    start_file = "debates2023-06-28d.xml"  # 
    output_dir = "data.xml_files"

    os.makedirs(output_dir, exist_ok=True)

    print(f"Connecting to {base_url} to get the file list...")

    try:
        response = requests.get(base_url)
        response.raise_for_status()  
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        all_xml_files = []
        for link in soup.find_all('a'):
            href = link.get('href')
            if href and href.endswith('.xml'):
                all_xml_files.append(href)
        
        print(f"Found {len(all_xml_files)} total XML files on the server.")

        try:
            start_index = all_xml_files.index(start_file)
            files_to_download = all_xml_files[start_index:]
            print(f"Found start file. Preparing to download {len(files_to_download)} files...")
        except ValueError:
            print(f"Error: Could not find the starting file '{start_file}' in the list.")
            print("Please check the file name and try again.")
            return

      
        for filename in tqdm(files_to_download, desc="Downloading files", unit="file"):
            file_url = urljoin(base_url, filename)
            local_path = os.path.join(output_dir, filename)

            if os.path.exists(local_path):
                continue
            
            try:
                file_response = requests.get(file_url)
                file_response.raise_for_status()
                
                with open(local_path, 'wb') as f:
                    f.write(file_response.content)
            
            except requests.exceptions.RequestException as e:
                print(f"Failed to download {filename}: {e}")

        print("\nDownload complete!")
        print(f"All files are saved in the '{output_dir}' directory.")

    except requests.exceptions.RequestException as e:
        print(f"Failed to access the website {base_url}: {e}")

if __name__ == "__main__":
    download_debate_files()

Connecting to https://www.theyworkforyou.com/pwdata/scrapedxml/debates/ to get the file list...
Found 19692 total XML files on the server.
Found start file. Preparing to download 956 files...



Download complete!
All files are saved in the 'xml_files' directory.


## Section B: Extract and Combine Text from Debate Files

So far we have downloaded the raw XML files. Now we will extract the text from them and combine them into single text files per date.
We will focus on extracting text from `<p>` tags only, ignoring redirect files.


In [3]:
import xml.etree.ElementTree as ET
import os
import re
from collections import defaultdict

XML_DIR = os.path.join('data', 'xml_files')
OUTPUT_DIR = os.path.join('data', 'combined_xml_files')
os.makedirs(OUTPUT_DIR, exist_ok=True)


def extract_text_from_xml(xml_file_path):
    all_text = []
    
    try:
        tree = ET.parse(xml_file_path)
        root = tree.getroot()

       # basically extract all <p> tags text

        for p_tag in root.findall('.//p'):
            if p_tag.text and p_tag.text.strip():
                all_text.append(p_tag.text.strip())

    except ET.ParseError as e:
        print(f"שגיאת ניתוח XML בקובץ {xml_file_path}: {e}")
        return ""
    except Exception as e:
        print(f"שגיאה כללית בקובץ {xml_file_path}: {e}")
        return ""
        
    return ' '.join(all_text)

# extract and combine texts
combined_texts = defaultdict(list)
file_list = os.listdir(XML_DIR)

print(f"נמצאו {len(file_list)} קבצים. מתחילים בחילוץ ואיחוד...")

for filename in file_list:
    if not filename.endswith('.xml'):
        continue
        
    file_path = os.path.join(XML_DIR, filename)
    
    match = re.search(r'debates(\d{4}-\d{2}-\d{2})[a-zA-Z]*d?\.xml$', filename)
    if not match:
        continue
        
    base_date = match.group(1)
    
    raw_text = extract_text_from_xml(file_path)
    
    if raw_text:
        combined_texts[base_date].append(raw_text)

print(f"סיום חילוץ. נמצאו {len(combined_texts)} תאריכים ייחודיים.")

# שמירת הקבצים המאוחדים
for date, text_list in combined_texts.items():
    final_combined_text = ' '.join(text_list)
    
    # שם הקובץ המאוחד: debates_YYYY-MM-DD.txt
    output_filename = f"debates_{date}.txt"
    output_path = os.path.join(OUTPUT_DIR, output_filename)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(final_combined_text)

print(f"הטקסטים המאוחדים (מבוססי <p>) נשמרו בהצלחה בספרייה **{OUTPUT_DIR}**")

נמצאו 956 קבצים. מתחילים בחילוץ ואיחוד...
סיום חילוץ. נמצאו 336 תאריכים ייחודיים.
הטקסטים המאוחדים (מבוססי <p>) נשמרו בהצלחה בספרייה **data\combined_xml_files**


## Section C1: Punctuation spacing using NLTK
in this section we will use NLTK's word_tokenize to properly space punctuation in the text files.
the nltk tokenizer is more sophisticated than simple regex and handles edge cases better,
though it may be less accurate than spaCy for complex texts.

In [ ]:
import os
from nltk.tokenize import word_tokenize

#
INPUT_DIR = os.path.join('data', 'combined_xml_files')
OUTPUT_DIR = os.path.join('data', 'tokenized_text_nltk') 
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"מתחיל ניקוי סימני פיסוק (עם NLTK). קבצים יישמרו ב- {OUTPUT_DIR}")

#
for filename in os.listdir(INPUT_DIR):
    if filename.endswith('.txt'):
        input_path = os.path.join(INPUT_DIR, filename)
        
        with open(input_path, 'r', encoding='utf-8') as f:
            raw_text = f.read()
            
        #tokenize the text by NLTK
        tokens = word_tokenize(raw_text)
        
        # join tokens back to string with spaces
        cleaned_text = ' '.join(tokens)
        output_path = os.path.join(OUTPUT_DIR, filename)
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(cleaned_text)

print("סיום ניקוי סימני פיסוק (NLTK).")

## Section C2: Punctuation spacing using spaCy 
This section uses spaCy for tokenization, which is generally more accurate and robust than NLTK's tokenizer.

In [10]:
import spacy
import os

INPUT_DIR = os.path.join('data', 'combined_xml_files')
OUTPUT_DIR = os.path.join('data', 'tokenized_text_spacy') # שם חדש
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("טוען את מודל השפה של spaCy...")
nlp = spacy.load("en_core_web_sm")
print("המודל נטען.")

print(f"מתחיל ניקוי סימני פיסוק (עם spaCy). קבצים יישמרו ב- {OUTPUT_DIR}")

# קריאה של כל הקבצים המאוחדים
for filename in os.listdir(INPUT_DIR):
    if filename.endswith('.txt'):
        input_path = os.path.join(INPUT_DIR, filename)
        
        with open(input_path, 'r', encoding='utf-8') as f:
            raw_text = f.read()
            
        
        doc = nlp(raw_text) # Spacy tokenization
        
        #for each token get the text
        tokens = [token.text for token in doc]
        
        cleaned_text = ' '.join(tokens)
        
        output_path = os.path.join(OUTPUT_DIR, filename)
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(cleaned_text)

print("סיום ניקוי סימני פיסוק (spaCy).")

טוען את מודל השפה של spaCy...
המודל נטען.
מתחיל ניקוי סימני פיסוק (עם spaCy). קבצים יישמרו ב- data\tokenized_text_spacy
סיום ניקוי סימני פיסוק (spaCy).


## Section D: Lemmatization using spaCy
So until now we have tokenized text files. In this section we will perform lemmatization using spaCy.

In [ ]:
import spacy
import os

INPUT_DIR = os.path.join('data', 'tokenized_text_spacy') 
OUTPUT_DIR = os.path.join('data', 'lemmatized_text')    
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("טוען את מודל השפה של spaCy (עם למטיזציה)...")

nlp = spacy.load("en_core_web_sm", disable=["ner"])
print("המודל נטען.")

# 2. הכנת רשימת הקבצים לעיבוד
file_paths = []
for filename in os.listdir(INPUT_DIR):
    if filename.endswith('.txt'):
        input_path = os.path.join(INPUT_DIR, filename)
        output_path = os.path.join(OUTPUT_DIR, filename)
        file_paths.append((input_path, output_path))

# 3. יצירת "גנרטור" שקורא את הקבצים
def text_generator(paths):
    for in_path, _ in paths:
        with open(in_path, 'r', encoding='utf-8') as f:
            yield f.read()

print(f"מתחיל עיבוד מהיר (batch) עם spaCy... מבצע למטיזציה על {len(file_paths)} קבצים.")

# 4. עיבוד כל הטקסטים ב-BATCH
processed_docs = nlp.pipe(text_generator(file_paths), batch_size=50)

# 5. שמירת התוצאות
for (in_path, out_path), doc in zip(file_paths, processed_docs):
    
    # חילוץ הלמה (lemma_) של כל טוקן
    # אנו גם ממירים הכל לאותיות קטנות (lowercase)
    lemmas = [token.lemma_.lower() for token in doc]
    lemmatized_text = ' '.join(lemmas)
    
    # שמירת הקובץ הנקי
    with open(out_path, 'w', encoding='utf-8') as f:
        f.write(lemmatized_text)

print(f"סיום עיבוד (Lemmatization). הקבצים נשמרו ב- {OUTPUT_DIR}")

## Section E: BM25 Vectorization without Optimizatios
right now we will create BM25 vectors for each document and save them as JSON files.
first we will filter out rare words that appear less than 5 times across all documents.
second we will compute the BM25 scores for each term in each document and save the non-zero scores in a JSON file.

This is the basic version without optimizations.
It computes BM25 scores for all terms in the vocabulary for each document,


In [65]:
import os
import glob
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from rank_bm25 import BM25Okapi
from collections import Counter
import nltk

nltk.download("punkt")
nltk.download("stopwords")

STOP_WORDS = set(stopwords.words("english"))

"""
load the texts from a folder, tokenize and clean them
"""
def load_texts(folder_path):
    docs, filenames = [], []
    for file_path in glob.glob(os.path.join(folder_path, "*.txt")):
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read().lower()
            tokens = [w for w in word_tokenize(text) if w.isalpha() and w not in STOP_WORDS]# remove punctuation and stop words
            docs.append(tokens) #
            filenames.append(os.path.basename(file_path))
    return docs, filenames

"""
filter out rare words that appear less than min_freq times across all documents
"""
def filter_rare_words(docs, min_freq=5):
    
    freq = Counter([word for doc in docs for word in doc])
    filtered_docs = [[w for w in doc if freq[w] >= min_freq] for doc in docs]
    return filtered_docs

"""
"""
def save_json_vectors(docs, filenames, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    bm25 = BM25Okapi(docs)
    #
    for doc_tokens, fname in zip(docs, filenames):
        vector = {}
        for term in bm25.idf.keys():
            f = doc_tokens.count(term)
            #
            if f > 0:
                score = bm25.idf[term] * ((f * (bm25.k1 + 1)) / (f + bm25.k1 * (1 - bm25.b + bm25.b * len(doc_tokens) / bm25.avgdl)))
                vector[term] = round(score, 3)
        with open(os.path.join(output_dir, fname.replace(".txt", ".json")), "w", encoding="utf-8") as f:
            json.dump(vector, f, ensure_ascii=False, indent=2)

base_data = "data"
base_model = "models"

# עיבוד גרסת spaCy
spacy_docs, spacy_names = load_texts(os.path.join(base_data, "tokenized_text_spacy"))
spacy_docs = filter_rare_words(spacy_docs)
save_json_vectors(spacy_docs, spacy_names, os.path.join(base_model, "bm25_word_json_dict"))

# עיבוד גרסת Lemmatized
lemm_docs, lemm_names = load_texts(os.path.join(base_data, "lemmatized_text"))
lemm_docs = filter_rare_words(lemm_docs)
save_json_vectors(lemm_docs, lemm_names, os.path.join(base_model, "bm25_lemm_json_dict"))

print("✅ Done! All vectors saved per document.")


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matanaminov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matanaminov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyboardInterrupt: 

## Section E: BM25 Vectorization with Rare Word Filtering and Optimizations
In this section we will optimize the BM25 vectorization process by implementing two key optimizations:
1. **Rare Word Filtering**: We will filter out rare words that appear less than a specified minimum frequency across all documents before computing BM25 scores. This reduces the vocabulary size and speeds up computations.
2. **Optimized Score Calculation**: Instead of iterating over the entire vocabulary for each document, we will only compute BM25 scores for the words that actually appear in each document. This significantly reduces the number of calculations needed, especially for large vocabularies.

In [48]:
import os
import glob
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from rank_bm25 import BM25Okapi
from collections import Counter
import nltk

nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

STOP_WORDS = set(stopwords.words("english"))

def load_texts(folder_path):
    docs, filenames = [], []
    print(f"טוען ומנקה טקסטים מ: {folder_path}")
    for file_path in glob.glob(os.path.join(folder_path, "*.txt")):
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read().lower()
            tokens = [w for w in word_tokenize(text) if w.isalpha() and w not in STOP_WORDS]
            docs.append(tokens)
            filenames.append(os.path.basename(file_path))
    return docs, filenames

def filter_rare_words(docs, min_freq=5):
    print("מסנן מילים נדירות...")
    freq = Counter([word for doc in docs for word in doc])
    # שומרים את אוצר המילים שאינו נדיר
    vocab = set(word for word, count in freq.items() if count >= min_freq)
    filtered_docs = [[w for w in doc if w in vocab] for doc in docs]
    print(f"גודל אוצר המילים המקורי: {len(freq)}, אחרי סינון: {len(vocab)}")
    return filtered_docs, vocab

def save_json_vectors_optimized(docs, filenames, output_dir, vocab):
    """
    גרסה מהירה: מחשבת BM25 רק עבור המילים שבאמת מופיעות בכל מסמך.
    """
    os.makedirs(output_dir, exist_ok=True)
    print(f"בונה מודל BM25 Okapi עבור {output_dir}...")
    bm25 = BM25Okapi(docs)
    
    print("יוצר ושומר וקטורים (בשיטה המהירה)...")
    for doc_tokens, fname in zip(docs, filenames):
        vector = {}
        
        # --- 1. האופטימיזציה ---
        # סופרים את המילים רק פעם אחת עבור המסמך הנוכחי
        doc_freqs = Counter(doc_tokens)
        
        # --- 2. האופטימיזציה ---
        # רצים *רק* על המילים הייחודיות במסמך זה (לא על כל המילון)
        for term in doc_freqs.keys():
            # (אין צורך ב-if f > 0 כי אנחנו יודעים שהמילה קיימת)
            f = doc_freqs[term]
            
            # --- זהירות: `rank_bm25` לא מחזיר ציון למילים שלא ב-IDF ---
            # אבל אנחנו סיננו את `docs` עם `vocab` אז זה אמור להיות בסדר
            if term not in bm25.idf:
                continue # מילה זו הייתה נדירה מדי והוסרה
                
            # אותה נוסחה בדיוק מהקוד שלך
            score = bm25.idf[term] * ((f * (bm25.k1 + 1)) / (f + bm25.k1 * (1 - bm25.b + bm25.b * len(doc_tokens) / bm25.avgdl)))
            vector[term] = round(score, 3)
            
        # שמירת קובץ ה-JSON
        with open(os.path.join(output_dir, fname.replace(".txt", ".json")), "w", encoding="utf-8") as f:
            json.dump(vector, f, ensure_ascii=False, indent=2)

# === הפעלת התהליך (עם האופטימיזציה) ===
base_data = "data"
base_model = "models"

# --- עיבוד גרסת spaCy (Word) ---
spacy_docs_raw, spacy_names = load_texts(os.path.join(base_data, "tokenized_text_spacy"))
spacy_docs_filtered, spacy_vocab = filter_rare_words(spacy_docs_raw)
save_json_vectors_optimized(spacy_docs_filtered, spacy_names, 
                            os.path.join(base_model, "bm25_word_json_dict"), 
                            spacy_vocab)
print("--- סיום עיבוד Word ---")


# --- עיבוד גרסת Lemmatized (Lemm) ---
lemm_docs_raw, lemm_names = load_texts(os.path.join(base_data, "lemmatized_files"))
lemm_docs_filtered, lemm_vocab = filter_rare_words(lemm_docs_raw)
save_json_vectors_optimized(lemm_docs_filtered, lemm_names, 
                            os.path.join(base_model, "bm25_lemm_json_dict"), 
                            lemm_vocab)
print("--- סיום עיבוד Lemm ---")

print("✅ Done! All optimized vectors saved per document.")

טוען ומנקה טקסטים מ: data/tokenized_text_spacy
מסנן מילים נדירות...
גודל אוצר המילים המקורי: 62557, אחרי סינון: 29989
בונה מודל BM25 Okapi עבור models/bm25_word_json_dict...
יוצר ושומר וקטורים (בשיטה המהירה)...
--- סיום עיבוד Word ---
טוען ומנקה טקסטים מ: data/lemmatized_files
מסנן מילים נדירות...
גודל אוצר המילים המקורי: 52714, אחרי סינון: 23604
בונה מודל BM25 Okapi עבור models/bm25_lemm_json_dict...
יוצר ושומר וקטורים (בשיטה המהירה)...
--- סיום עיבוד Lemm ---
✅ Done! All optimized vectors saved per document.


## Section F: Word2Vec Document Vectors with Optimizations
In this section we will create Word2Vec document vectors with optimizations.
The optimizations include:
1. **Multi-core Processing**: We will utilize all available CPU cores to speed up the training of the Word2Vec model by setting the `workers` parameter.
2. **Efficient Document Vector Calculation**: We will compute document vectors by averaging only the word vectors of words that exist in the Word2Vec vocabulary, avoiding unnecessary computations for out-of-vocabulary words.

In [7]:
import os  # 🚀 הוספנו את זה
import glob
import json
import re
import sys
import subprocess
import importlib.util

# --- Auto-install gensim if not found ---
if importlib.util.find_spec("gensim") is None:
    print("Installing gensim ...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gensim"])

# Now safe to import
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import nltk

# --- Downloads for NLTK ---
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)
STOP_WORDS = set(stopwords.words("english"))

# --- Utility functions ---

def clean_text(text):
    """Remove punctuation, digits, quotes, etc."""
    text = re.sub(r"[^\w\s]", " ", text)  # remove punctuation
    text = re.sub(r"\d+", " ", text)      # remove numbers
    text = text.lower().strip()
    return text

def load_docs(folder, remove_stop=False):
    """Load all txt files and tokenize"""
    docs, names = [], []
    for path in glob.glob(os.path.join(folder, "*.txt")):
        with open(path, "r", encoding="utf-8") as f:
            text = clean_text(f.read())
            tokens = [w for w in word_tokenize(text) if w.isalpha()]
            if remove_stop:
                tokens = [w for w in tokens if w not in STOP_WORDS]
            docs.append(tokens)
            names.append(os.path.basename(path))
    return docs, names

def build_doc_vectors(docs, model):
    """Average all word vectors in each document"""
    vectors = []
    for tokens in docs:
        valid = [w for w in tokens if w in model.wv]
        if valid:
            vecs = np.array([model.wv[w] for w in valid])
            vectors.append(np.mean(vecs, axis=0))
        else:
            vectors.append(np.zeros(model.vector_size))
    return vectors

def save_vectors(vectors, names, out_dir):
    """Save each document vector as its own JSON"""
    os.makedirs(out_dir, exist_ok=True)
    for vec, name in zip(vectors, names):
        with open(os.path.join(out_dir, name.replace(".txt", ".json")), "w", encoding="utf-8") as f:
            json.dump({"vector": vec.tolist()}, f, ensure_ascii=False, indent=2)

# --- Main Configuration ---
base_data = "data"
base_model = "models"
VECTOR_SIZE = 300

# --- 🚀 אופטימיזציה: שימוש בכל הליבות הזמינות ---
CPU_CORES = os.cpu_count() or 1 # ( or 1 למקרה שהפקודה נכשלת)
print(f"Word2Vec | מפעיל אופטימיזציה: משתמש ב-{CPU_CORES} ליבות CPU.")


# --- 1️⃣ tokenized_text_spacy - with stopwords ---
print("\nמתחיל מודל 1/4: Word (עם stop-words)")
docs, names = load_docs(os.path.join(base_data, "tokenized_text_spacy"), remove_stop=False)
model = Word2Vec(sentences=docs, vector_size=VECTOR_SIZE, window=5, min_count=2, workers=CPU_CORES) # 🚀
save_vectors(build_doc_vectors(docs, model), names, os.path.join(base_model, "w2v_word_with_stop"))

# --- 2️⃣ tokenized_text_spacy - no stopwords ---
print("\nמתחיל מודל 2/4: Word (בלי stop-words)")
docs_ns, names_ns = load_docs(os.path.join(base_data, "tokenized_text_spacy"), remove_stop=True)
model_ns = Word2Vec(sentences=docs_ns, vector_size=VECTOR_SIZE, window=5, min_count=2, workers=CPU_CORES) # 🚀
save_vectors(build_doc_vectors(docs_ns, model_ns), names_ns, os.path.join(base_model, "w2v_word_no_stop"))

# --- 3️⃣ lemmatized_text - with stopwords ---
print("\nמתחיל מודל 3/4: Lemma (עם stop-words)")
lemm_docs, lemm_names = load_docs(os.path.join(base_data, "lemmatized_files"), remove_stop=False)
model_lemm = Word2Vec(sentences=lemm_docs, vector_size=VECTOR_SIZE, window=5, min_count=2, workers=CPU_CORES) # 🚀
save_vectors(build_doc_vectors(lemm_docs, model_lemm), lemm_names, os.path.join(base_model, "w2v_lemm_with_stop"))

# --- 4️⃣ lemmatized_text - no stopwords ---
print("\nמתחיל מודל 4/4: Lemma (בלי stop-words)")
lemm_docs_ns, lemm_names_ns = load_docs(os.path.join(base_data, "lemmatized_files"), remove_stop=True)
model_lemm_ns = Word2Vec(sentences=lemm_docs_ns, vector_size=VECTOR_SIZE, window=5, min_count=2, workers=CPU_CORES) # 🚀
save_vectors(build_doc_vectors(lemm_docs_ns, model_lemm_ns), lemm_names_ns, os.path.join(base_model, "w2v_lemm_no_stop"))

print("\n✅ Done! 4 folders with 300-dimensional Word2Vec document vectors created.")

Word2Vec | מפעיל אופטימיזציה: משתמש ב-8 ליבות CPU.

מתחיל מודל 1/4: Word (עם stop-words)

מתחיל מודל 2/4: Word (בלי stop-words)

מתחיל מודל 3/4: Lemma (עם stop-words)


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'



מתחיל מודל 4/4: Lemma (בלי stop-words)


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'



✅ Done! 4 folders with 300-dimensional Word2Vec document vectors created.


## Section G: SimCSE Document Vectors
here we will create document vectors using the SimCSE model from the SentenceTransformers library.
what it does is that it takes in sentences and produces high-quality sentence embeddings that capture semantic meaning.
its different from traditional methods like TF-IDF or Word2Vec because it uses deep learning to understand context.

In [ ]:
import os
import glob
import json
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

print("--- התחלת סעיף ג': SimCSE ---")

# --- 1. הגדרת נתיבים ---
INPUT_DIR = os.path.join('data', 'combined_xml_files') 
OUTPUT_DIR = os.path.join('models', 'simcse_origen')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- 2. בדיקת האצת GPU (MPS) עבור M1/M2 ---
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("זיהוי M1/M2 GPU (MPS). מפעיל האצה.")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("זיהוי NVIDIA GPU (CUDA). מפעיל האצה.")
else:
    device = torch.device("cpu")
    print("לא זוהתה האצת GPU. משתמש ב-CPU.")

# --- 3. טעינת מודל SimCSE ---
model_name = 'princeton-nlp/unsup-simcse-bert-base-uncased'
print(f"טוען את המודל {model_name}...")
model = SentenceTransformer(model_name, device=device)
print("המודל נטען.")

# --- 4. טעינת המסמכים ---
all_texts = []
all_names = []
print(f"טוען את כל קבצי המקור מ-{INPUT_DIR}...")
for file_path in glob.glob(os.path.join(INPUT_DIR, "*.txt")):
    with open(file_path, "r", encoding="utf-8") as f:
        all_texts.append(f.read())
    all_names.append(os.path.basename(file_path))
print(f"נמצאו {len(all_texts)} מסמכים.")

# --- 5. יצירת וקטורים (Embeddings) ---
print("מתחיל ביצירת וקטורי SimCSE...")
vectors = model.encode(
    all_texts, 
    show_progress_bar=True, 
    batch_size=32, # אפשר להגדיל אם יש לך הרבה VRAM
    convert_to_numpy=True 
)
print("יצירת הוקטורים הושלמה.")

# --- 6. שמירת הוקטורים ---
print(f"שומר את הוקטורים ב-{OUTPUT_DIR}...")
for vec, name in zip(vectors, all_names):
    output_path = os.path.join(OUTPUT_DIR, name.replace(".txt", ".json"))
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump({"vector": vec.tolist()}, f, ensure_ascii=False, indent=2)

print("✅ סיום סעיף ג'! וקטורי SimCSE נשמרו.")

## Section H: SBERT Document Vectors
here we will create document vectors using the SBERT (Sentence-BERT) model from the SentenceTransformers library.
it works similarly to SimCSE but is trained specifically for producing high-quality sentence embeddings.
it differs from simCSE in that it uses a siamese network architecture to directly optimize for sentence similarity tasks.
siamese networks consist of two identical subnetworks that share weights and are trained to minimize the distance between similar sentences while maximizing the distance between dissimilar ones.

In [2]:
import os
import glob
import json
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

print("--- התחלת סעיף ד': SBERT ---")

# --- 1. הגדרת נתיבים ---
# שימוש באותם קבצי מקור כמו SimCSE
INPUT_DIR = os.path.join('data', 'combined_xml_files') 
# תיקיית פלט לפי שם הקבוצה במטלה
OUTPUT_DIR = os.path.join('models', 'sbert_origen')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- 2. בדיקת האצת GPU (MPS) ---
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("זיהוי M1/M2 GPU (MPS). מפעיל האצה.")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("זיהוי NVIDIA GPU (CUDA). מפעיל האצה.")
else:
    device = torch.device("cpu")
    print("לא זוהתה האצת GPU. משתמש ב-CPU.")

# --- 3. טעינת מודל SBERT ---
# זהו מודל SBERT פופולרי ומומלץ לשימוש כללי
model_name = 'all-mpnet-base-v2'
print(f"טוען את המודל {model_name}...")
model = SentenceTransformer(model_name, device=device)
print("המודל נטען.")

# --- 4. טעינת המסמכים ---
# הקוד זהה לסעיף הקודם
all_texts = []
all_names = []
print(f"טוען את כל קבצי המקור מ-{INPUT_DIR}...")
for file_path in glob.glob(os.path.join(INPUT_DIR, "*.txt")):
    with open(file_path, "r", encoding="utf-8") as f:
        all_texts.append(f.read())
    all_names.append(os.path.basename(file_path))
print(f"נמצאו {len(all_texts)} מסמכים.")

# --- 5. יצירת וקטורים (Embeddings) ---
print("מתחיל ביצירת וקטורי SBERT...")
vectors = model.encode(
    all_texts, 
    show_progress_bar=True, 
    batch_size=32,
    convert_to_numpy=True 
)
print("יצירת הוקטורים הושלמה.")

# --- 6. שמירת הוקטורים ---
print(f"שומר את הוקטורים ב-{OUTPUT_DIR}...")
for vec, name in zip(vectors, all_names):
    output_path = os.path.join(OUTPUT_DIR, name.replace(".txt", ".json"))
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump({"vector": vec.tolist()}, f, ensure_ascii=False, indent=2)

print("✅ סיום סעיף ד'! וקטורי SBERT נשמרו.")

--- התחלת סעיף ד': SBERT ---
זיהוי M1/M2 GPU (MPS). מפעיל האצה.
טוען את המודל all-mpnet-base-v2...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

המודל נטען.
טוען את כל קבצי המקור מ-data/combined_xml_files...
נמצאו 336 מסמכים.
מתחיל ביצירת וקטורי SBERT...


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

יצירת הוקטורים הושלמה.
שומר את הוקטורים ב-models/sbert_origen...
✅ סיום סעיף ד'! וקטורי SBERT נשמרו.


## Section I1: we used kmeans with the best k  + BM25 Features for Advanced Analysis with mutual_info_classif and Chi2 and gini to find important features



In [82]:
import os
import glob
import json
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import mutual_info_classif, chi2
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import numpy as np

# --- 1. תצורה ---
print("--- מתחיל פתרון 1: SVD + K-Means ---")

# !!! שנה את הנתיב בין ריצות
TFIDF_DIR = os.path.join('models', 'bm25_word_json_dict')
#TFIDF_DIR = os.path.join('models', 'bm25_lemm_json_dict')

# נצמצם את 19,000+ המימדים ל-50 "מושגים"
N_SVD_COMPONENTS = 50

# במקום Silhouette נבחר K ידני. תשנה לפי מה שמעניין אותך.
BEST_K = 5

print(f"מעבד מטריצה: {TFIDF_DIR}")
print(f"יצמצם מימדים ל- {N_SVD_COMPONENTS} רכיבים.")
print(f"משתמש ב-K קבוע: {BEST_K}")

# --- 2. טעינת נתונים ---
doc_vectors = []
filenames = []
print("טוען קבצי JSON...")
for file_path in glob.glob(os.path.join(TFIDF_DIR, "*.json")):
    fname = os.path.basename(file_path)
    with open(file_path, 'r', encoding='utf-8') as f:
        doc_vectors.append(json.load(f))
    filenames.append(fname)
print(f"נמצאו {len(doc_vectors)} מסמכים.")

# --- 3. בניית מטריצה ---
print("ממיר ל-feature matrix (X_sparse)...")
vectorizer = DictVectorizer(sparse=True)
X_sparse = vectorizer.fit_transform(doc_vectors)
feature_names = vectorizer.get_feature_names_out()
print(f"מטריצה דלילה נוצרה בגודל: {X_sparse.shape}")

# --- 4. צמצום מימדים עם SVD ---
print(f"מבצע Truncated SVD לצמצום מימדים ל- {N_SVD_COMPONENTS} רכיבים...")
svd = TruncatedSVD(n_components=N_SVD_COMPONENTS, random_state=42)
X_reduced = svd.fit_transform(X_sparse)
print(f"מטריצה מצומצמת נוצרה בגודל: {X_reduced.shape}")

# --- 5. הרצה של K-Means עם K קבוע ---
print(f"\n--- 5. הרצה של K-Means עם K={BEST_K} ---")
kmeans = KMeans(n_clusters=BEST_K, random_state=42, n_init=10)
y = kmeans.fit_predict(X_reduced)
cluster_counts = np.bincount(y)
print(f"התפלגות סופית ב- {BEST_K} קבוצות:")
print(cluster_counts)

# --- 6. ניתוח חשיבות פיצ'רים (מול X_sparse המקורי) ---
print(f"\n--- 6. חישוב חשיבות פיצ'רים (מול X_sparse המקורי) ---")

print(f"[K={BEST_K}] מחשב Information Gain...")
ig_results = pd.DataFrame()
try:
    print("  (ממיר לצפוף...)")
    X_dense = X_sparse.toarray()
    ig_scores = mutual_info_classif(X_dense, y, discrete_features=False)
    ig_results = pd.DataFrame({'feature': feature_names, 'info_gain': ig_scores})
    ig_results = ig_results.sort_values(by='info_gain', ascending=False)
    print("  חישוב Info Gain הצליח.")
except MemoryError:
    print("  !!! שגיאת זיכרון. מדלג על Info Gain.")
except Exception as e:
    print(f"  !!! שגיאה ב-Info Gain: {e}. מדלג.")

print(f"[K={BEST_K}] מחשב Chi-squared...")
chi2_scores, p_values = chi2(X_sparse, y)
chi2_results = pd.DataFrame({'feature': feature_names,
                             'chi2_score': chi2_scores,
                             'p_value': p_values})
chi2_results = chi2_results.sort_values(by='chi2_score', ascending=False)
print("  חישוב Chi-squared הצליח.")

# --- 7. שמירת התוצאות לאקסל ---
output_filename = f"first_method_word_{os.path.basename(TFIDF_DIR)}.xlsx"
print(f"\n--- 7. שומר הכל לקובץ אקסל אחד ---")
print(f"קובץ: {output_filename}")

with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
    # גיליונות מלאים
    if not ig_results.empty:
        ig_results.to_excel(writer, sheet_name='Information Gain (Full)', index=False)
    chi2_results.to_excel(writer, sheet_name='Chi-squared (Full)', index=False)

    # טופ 20
    if not ig_results.empty:
        ig_results.head(20).to_excel(writer, sheet_name='Top 20 Info Gain', index=False)
    chi2_results.head(20).to_excel(writer, sheet_name='Top 20 Chi-squared', index=False)

print("\n\n✅✅✅ סיום פתרון 1 (ללא Silhouette, ללא גרפים, ללא Gini)! ✅✅✅")

--- מתחיל פתרון 1: SVD + K-Means ---
מעבד מטריצה: models/bm25_word_json_dict
יצמצם מימדים ל- 50 רכיבים.
משתמש ב-K קבוע: 5
טוען קבצי JSON...
נמצאו 336 מסמכים.
ממיר ל-feature matrix (X_sparse)...
מטריצה דלילה נוצרה בגודל: (336, 29989)
מבצע Truncated SVD לצמצום מימדים ל- 50 רכיבים...
מטריצה מצומצמת נוצרה בגודל: (336, 50)

--- 5. הרצה של K-Means עם K=5 ---
התפלגות סופית ב- 5 קבוצות:
[ 47  11  52 225   1]

--- 6. חישוב חשיבות פיצ'רים (מול X_sparse המקורי) ---
[K=5] מחשב Information Gain...
  (ממיר לצפוף...)
  חישוב Info Gain הצליח.
[K=5] מחשב Chi-squared...
  חישוב Chi-squared הצליח.

--- 7. שומר הכל לקובץ אקסל אחד ---
קובץ: first_method_word_bm25_word_json_dict.xlsx


✅✅✅ סיום פתרון 1 (ללא Silhouette, ללא גרפים, ללא Gini)! ✅✅✅


## Section I2: NMF Topic Modeling with BM25 Features

In [84]:
import os
import glob
import json
import pandas as pd
import numpy as np

# Scikit-Learn Imports
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import NMF

# --- 1. Configuration ---
print("--- Starting NMF Topic Modeling (simplified) ---")

# !!! UPDATE THIS PATH to point to your JSON folder
TFIDF_DIR = os.path.join('models', 'bm25_lemm_json_dict')

# Parameters
BEST_K = 5         # מספר הנושאים (topics) – תקבע ידנית
N_TOP_WORDS = 20     # מספר המילים לכל topic

print(f"Target Directory: {TFIDF_DIR}")
print(f"Using fixed K (Topics): {BEST_K}")

# --- 2. Load Data ---
doc_vectors = []
filenames = []

print("Loading JSON files...")
json_files = glob.glob(os.path.join(TFIDF_DIR, "*.json"))

if not json_files:
    raise FileNotFoundError(f"No JSON files found in {TFIDF_DIR}")

for file_path in json_files:
    fname = os.path.basename(file_path)
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        doc_vectors.append(data)
    filenames.append(fname)

print(f"Successfully loaded {len(doc_vectors)} documents.")

# --- 3. Build Feature Matrix ---
print("Vectorizing data...")
vectorizer = DictVectorizer(sparse=True)
X_sparse = vectorizer.fit_transform(doc_vectors)
feature_names = vectorizer.get_feature_names_out()
print(f"Matrix Shape: {X_sparse.shape} (Docs x Words)")

# --- 4. Fix for NMF: Handle Negative Values ---
min_val = X_sparse.data.min() if X_sparse.nnz > 0 else 0
if min_val < 0:
    print(f"!!! Warning: Negative values detected (Min: {min_val:.4f}). Shifting data...")
    X_sparse.data -= min_val

# --- 5. Train Final NMF Model with Fixed K ---
print(f"\n--- Training Final NMF Model with K={BEST_K} ---")
nmf_model = NMF(
    n_components=BEST_K,
    random_state=42,
    init='nndsvda',
    max_iter=500
)

# W: Document-Topic Matrix (Docs x Topics)
W = nmf_model.fit_transform(X_sparse)

# H: Topic-Word Matrix (Topics x Words)
H = nmf_model.components_

# Assign each document to its dominant topic
doc_topic_labels = np.argmax(W, axis=1)
topic_counts = np.bincount(doc_topic_labels)

# --- 6. Extract Topics & Keywords ---
print("Extracting keywords per topic...")
topic_dict = {}

for topic_idx, topic_weights in enumerate(H):
    # indices of top N words
    top_indices = topic_weights.argsort()[:-N_TOP_WORDS - 1:-1]

    top_words = [feature_names[i] for i in top_indices]
    top_scores = [topic_weights[i] for i in top_indices]

    # quick console summary
    print(f"Topic {topic_idx} ({topic_counts[topic_idx]} docs): {', '.join(top_words[:5])}...")

    # store for Excel
    topic_dict[f'Topic_{topic_idx}_Words'] = top_words
    topic_dict[f'Topic_{topic_idx}_Weights'] = top_scores

topics_df = pd.DataFrame(topic_dict)

# --- 7. Build Doc Mapping DataFrame ---
mapping_df = pd.DataFrame({
    'Filename': filenames,
    'Predicted_Topic': doc_topic_labels,
    'Topic_Strength': np.max(W, axis=1)  # how strongly the doc fits its main topic
})

# --- 8. Save to Excel (no plots) ---
output_filename = f"Second_method_lemmas_NMF_Results_.xlsx"
print(f"\n--- Saving results to: {output_filename} ---")

with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
    # Sheet 1: Topic Keywords
    topics_df.to_excel(writer, sheet_name='Topic Keywords', index=False)

    # Sheet 2: Doc Mapping
    mapping_df.to_excel(writer, sheet_name='Doc Mapping', index=False)

print("✅ Processing Complete (no Silhouette search, no plots)!")

--- Starting NMF Topic Modeling (simplified) ---
Target Directory: models/bm25_lemm_json_dict
Using fixed K (Topics): 5
Loading JSON files...
Successfully loaded 336 documents.
Vectorizing data...
Matrix Shape: (336, 23604) (Docs x Words)

--- Training Final NMF Model with K=5 ---
Extracting keywords per topic...
Topic 0 (190 docs): tax, economy, government, budget, business...
Topic 1 (8 docs): birthplace, championship, fc, cricket, cathedral...
Topic 2 (43 docs): ally, military, civilian, humanitarian, unrwa...
Topic 3 (89 docs): amendment, clause, legislation, hon, bill...
Topic 4 (6 docs): tapp, antonia, goldsborough, ayoub, sackman...

--- Saving results to: Second_method_lemmas_NMF_Results_.xlsx ---
✅ Processing Complete (no Silhouette search, no plots)!


In [89]:
import os
import json
import math
from collections import Counter, defaultdict

import pandas as pd  # לשמירה לאקסל
from sklearn.feature_extraction import DictVectorizer
from sklearn.cluster import KMeans

# ===== הגדרות שתתאים לסביבה שלך =====

BM25_DIR = "models/bm25_word_json_dict"  # או bm25_word_json_dict
N_CLUSTERS = 10                          # מספר הקלאסטרים ל-KMeans
MIN_DOCS_PER_TERM = 5                    # מינימום מסמכים למילה כדי לחשב לה IG
TOP_K = 100                              # כמה מילים עם IG הכי גבוה להציג
OUT_IG_XLSX = "models/third_method_words_term_information_gain_clusters.xlsx"  # רק אקסל


def load_bm25_vectors(bm25_dir):
    """
    טוען את כל קבצי ה-BM25 JSON מהתיקייה.
    מחזיר:
        doc_vectors: dict {doc_id -> {term: score}}
    """
    doc_vectors = {}
    file_count = 0

    for fname in os.listdir(bm25_dir):
        if not fname.endswith(".json"):
            continue
        doc_id = os.path.splitext(fname)[0]
        full_path = os.path.join(bm25_dir, fname)
        with open(full_path, "r", encoding="utf-8") as f:
            vec = json.load(f)
        doc_vectors[doc_id] = {t: float(s) for t, s in vec.items()}
        file_count += 1

    print(f"טענו {file_count} מסמכים עם BM25 מ: {bm25_dir}")
    return doc_vectors


def cluster_documents(doc_vectors, n_clusters=10):
    """
    עושה קלאסטרינג למסמכים באמצעות KMeans על וקטורי BM25.
    מחזיר:
        doc_labels: dict {doc_id -> 'cluster_X'}
    """
    doc_ids = list(doc_vectors.keys())
    vecs = list(doc_vectors.values())

    # DictVectorizer הופך רשימת dict-ים למטריצה sparse
    dv = DictVectorizer(sparse=True)
    X = dv.fit_transform(vecs)

    print(f"מטריצת מסמכים: {X.shape[0]} מסמכים, {X.shape[1]} פיצ'רים (מילים).")
    print(f"מריץ KMeans עם {n_clusters} קלאסטרים...")

    kmeans = KMeans(
        n_clusters=n_clusters,
        random_state=42,
        n_init=10
    )
    cluster_ids = kmeans.fit_predict(X)

    doc_labels = {}
    for doc_id, cid in zip(doc_ids, cluster_ids):
        doc_labels[doc_id] = f"cluster_{cid}"

    # קצת סטטיסטיקה
    label_counts = Counter(doc_labels.values())
    print("התפלגות קלאסטרים:")
    for lbl, cnt in label_counts.items():
        print(f"  {lbl}: {cnt} מסמכים")

    return doc_labels


def compute_entropy(label_counts, total_docs):
    """
    H(Y) = - Σ p(y) log2 p(y)
    """
    entropy = 0.0
    for count in label_counts.values():
        if count == 0:
            continue
        p = count / total_docs
        entropy -= p * math.log2(p)
    return entropy


def compute_information_gain_from_clusters(doc_vectors, doc_labels,
                                           min_docs_per_term=5):
    """
    מחשב Information Gain לכל מונח ביחס לקלאסטרים (labels).
    doc_vectors: dict {doc_id -> {term: score}}
    doc_labels: dict {doc_id -> label}  (לייבל = cluster_X)
    מחזיר:
        term_ig: dict {term -> IG}
    """

    # מכינים רשימה של (doc_id, terms_set, label)
    docs = []
    for doc_id, vec in doc_vectors.items():
        if doc_id not in doc_labels:
            continue
        terms = set(vec.keys())
        label = doc_labels[doc_id]
        docs.append((doc_id, terms, label))

    total_docs = len(docs)
    print(f"סך הכל מסמכים עם BM25+קלאסטר: {total_docs}")

    # התפלגות כללית של הלייבלים (הקלאסטרים)
    label_counts = Counter(label for _, _, label in docs)

    base_entropy = compute_entropy(label_counts, total_docs)
    print(f"H(Y) (אנטרופיית קלאסטרים): {base_entropy:.4f}")

    # term_label_presence[term][label] = כמה מסמכים עם המונח ועם ה-label
    term_label_presence = defaultdict(Counter)
    # term_doc_count[term] = כמה מסמכים שבהם המונח מופיע
    term_doc_count = Counter()

    for _, terms, label in docs:
        for term in terms:
            term_label_presence[term][label] += 1
            term_doc_count[term] += 1

    term_ig = {}

    for term, present_count in term_doc_count.items():
        if present_count < min_docs_per_term:
            continue

        N1 = present_count        # מספר מסמכים עם המונח (X=1)
        N0 = total_docs - N1      # מספר מסמכים בלי המונח (X=0)

        if N0 == 0 or N1 == 0:
            continue

        # H(Y | X=1)
        entropy_y_given_x1 = 0.0
        for label, label_total_count in label_counts.items():
            N11 = term_label_presence[term][label]
            if N11 == 0:
                continue
            p_y_given_x1 = N11 / N1
            entropy_y_given_x1 -= p_y_given_x1 * math.log2(p_y_given_x1)

        # H(Y | X=0)
        entropy_y_given_x0 = 0.0
        for label, label_total_count in label_counts.items():
            N11 = term_label_presence[term][label]
            N01 = label_total_count - N11  # מסמכים עם label אבל בלי term
            if N01 == 0:
                continue
            p_y_given_x0 = N01 / N0
            entropy_y_given_x0 -= p_y_given_x0 * math.log2(p_y_given_x0)

        p_x1 = N1 / total_docs
        p_x0 = N0 / total_docs

        cond_entropy = p_x1 * entropy_y_given_x1 + p_x0 * entropy_y_given_x0
        ig = base_entropy - cond_entropy
        term_ig[term] = ig

    return term_ig


def main():
    # 1. טוענים BM25
    doc_vectors = load_bm25_vectors(BM25_DIR)

    # 2. מריצים קלאסטרינג ומפיקים pseudo-labels
    doc_labels = cluster_documents(doc_vectors, n_clusters=N_CLUSTERS)

    # 3. מחשבים Information Gain לכל מונח ביחס לקלאסטרים
    term_ig = compute_information_gain_from_clusters(
        doc_vectors,
        doc_labels,
        min_docs_per_term=MIN_DOCS_PER_TERM
    )
    print(f"נמצאו {len(term_ig)} מילים עם IG אחרי סינון.")

    # 4. מיון והצגת TOP K
    sorted_terms = sorted(term_ig.items(), key=lambda x: x[1], reverse=True)

    print(f"\nTop {TOP_K} מילים לפי Information Gain (קלאסטרים כ-labels):\n")
    for term, ig in sorted_terms[:TOP_K]:
        print(f"{term:25s}  IG = {ig:.5f}")

    # 5. שמירה רק ל-Excel
    os.makedirs(os.path.dirname(OUT_IG_XLSX), exist_ok=True)
    df = pd.DataFrame(sorted_terms, columns=["term", "information_gain"])
    df.to_excel(OUT_IG_XLSX, index=False)
    print(f"\n📊 שמרתי את תוצאות ה-IG לאקסל ב: {OUT_IG_XLSX}")


if __name__ == "__main__":
    main()

טענו 336 מסמכים עם BM25 מ: models/bm25_word_json_dict
מטריצת מסמכים: 336 מסמכים, 29989 פיצ'רים (מילים).
מריץ KMeans עם 10 קלאסטרים...
התפלגות קלאסטרים:
  cluster_0: 150 מסמכים
  cluster_3: 172 מסמכים
  cluster_4: 1 מסמכים
  cluster_2: 7 מסמכים
  cluster_8: 1 מסמכים
  cluster_1: 1 מסמכים
  cluster_6: 1 מסמכים
  cluster_9: 1 מסמכים
  cluster_5: 1 מסמכים
  cluster_7: 1 מסמכים
סך הכל מסמכים עם BM25+קלאסטר: 336
H(Y) (אנטרופיית קלאסטרים): 1.3051
נמצאו 26734 מילים עם IG אחרי סינון.

Top 100 מילים לפי Information Gain (קלאסטרים כ-labels):

intent                     IG = 0.10532
nationals                  IG = 0.10335
damaging                   IG = 0.09814
wage                       IG = 0.09279
consumers                  IG = 0.09157
deter                      IG = 0.09108
buy                        IG = 0.09102
inflation                  IG = 0.09093
drafted                    IG = 0.08850
budget                     IG = 0.08668
roof                       IG = 0.08633
diplomatic            